In [ ]:
import win32com.client
from datetime import datetime
import pytz
import os


def find_folders(outlook, folder_name):
    """Recherche récursive d'un dossier Outlook (par nom) dans toutes les boîtes."""
    found_folders = []

    def search(folder, level=0):
        indent = "    " * level
        print(f"{indent}📂 {folder.Name}")

        if folder.Name.lower() == folder_name.lower():
            print(f"{indent}✅ Dossier trouvé : {folder.FolderPath}")
            found_folders.append(folder)

        for subfolder in folder.Folders:
            search(subfolder, level + 1)

    for store in outlook.Folders:
        print(f"\n=== Boîte : {store.Name} ===")
        search(store)

    return found_folders


def search_outlook_mails(folder_name, sender=None, subject=None,
                         date_after=None, date_before=None, attachments_dir=None):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    results = []

    target_folders = find_folders(outlook, folder_name)

    if not target_folders:
        print(f"⚠️ Aucun dossier nommé '{folder_name}' trouvé.")
        return []

    for folder in target_folders:
        print(f"\n📌 Analyse du dossier : {folder.FolderPath}")

        items = folder.Items
        items.Sort("[ReceivedTime]", True)

        # Restrict uniquement sur la période
        restriction = []
        if date_after:
            restriction.append(f"[ReceivedTime] >= '{date_after.strftime('%m/%d/%Y %I:%M %p')}'")
        if date_before:
            restriction.append(f"[ReceivedTime] <= '{date_before.strftime('%m/%d/%Y %I:%M %p')}'")

        if restriction:
            query = " AND ".join(restriction)
            print(f"🔎 Restrict appliqué : {query}")
            items = items.Restrict(query)

        count = 0
        for item in items:
            if item.Class != 43:  # pas un MailItem
                continue

            match = True

            # Filtrage Python (expéditeur et objet en LIKE)
            if sender and sender.lower() not in (item.SenderEmailAddress or "").lower():
                match = False
            if subject and subject.lower() not in (item.Subject or "").lower():
                match = False

            if match:
                count += 1
                mail_info = {
                    "Sender": item.SenderName,
                    "Email": item.SenderEmailAddress,
                    "Subject": item.Subject,
                    "Date": item.ReceivedTime.strftime("%Y-%m-%d %H:%M:%S"),
                    "Folder": folder.FolderPath
                }

                if attachments_dir:
                    os.makedirs(attachments_dir, exist_ok=True)
                    for attachment in item.Attachments:
                        try:
                            save_path = os.path.join(attachments_dir, attachment.FileName)
                            attachment.SaveAsFile(save_path)
                            print(f"📎 Pièce jointe sauvegardée : {save_path}")
                        except Exception as e:
                            print(f"⚠️ Erreur sauvegarde PJ : {e}")

                results.append(mail_info)

        print(f"✅ {count} mails trouvés dans {folder.FolderPath}")

    return results


if __name__ == "__main__":
    tz = pytz.timezone("UTC")
    date_after = tz.localize(datetime(2025, 1, 8))
    date_before = tz.localize(datetime(2025, 2, 8))

    mails = search_outlook_mails(
        folder_name="compliance",                   # 🔹 Nom du dossier exact
        sender="compliance_eme@gos.orange.com",             # 🔹 LIKE → sous-chaîne acceptée
        subject="Liste alertes kyc ",                        # 🔹 LIKE → sous-chaîne acceptée
        date_after=date_after,
        date_before=date_before,
        attachments_dir=r"C:\Users\MLSD0903\OneDrive - orange.com\Images\ProjetGetMail\extractions"
    )

    print(f"\n--- {len(mails)} mails trouvés ---")
    for mail in mails:
        print(f"[{mail['Date']}] {mail['Sender']} ({mail['Email']}) - {mail['Subject']} (Dossier: {mail['Folder']})")
